In [ ]:
%load_ext autoreload    
%autoreload 2    



In [ ]:
import os 
import sys
from pathlib import Path # To set data downloading path

# Append ghostcoder folder to path 
sys.path.append(os.path.abspath('..'))
import ghostcoder

# For visualize the graph
from IPython.display import Image, display

In [ ]:

os.environ["TAVILY_API_KEY"] = "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"

bgi_key = 'sk-NYPE5WNGTK781Nox801a5934A9F84a03BfC525359eBaE7B6'
bgi_base = 'http://10.224.28.80:3000/v1'

openai_key = "OYXxhyOyPDsJNABxugxjJJxntQ0B1EB0"
openai_base = "https://api3.aitok.ai/openai/v1"

openai_chat_model = 'gpt-4o'
openai_code_model = 'gpt-4o'

Here we will test and illustrate each subgraph in Ghostcoder, as an important component of BIA (bioinformatics agnet), mainly functions to complete the generation and execution of bioinformatics analysis codes. It contains five subgraphs. They are filemanager, retriever, coder and webcrawler executor respectively.


### Omics data preparation

In [ ]:
#──────work_dir 
#  └───data
#    └─Input_data.whatever
# First lets download a scRNAseq data
import scanpy as sc

from ghostcoder.config import file_config

current_dir = Path.cwd()

# Ghostcoder pre-set WORK_DIR and INPUT_DATA_DIR for continues bioinformatics tasks using one input data

# Download scRNAseq data
sc.settings.datasetdir = current_dir/ file_config.INPUT_DATA_DIR # Download data into data/ folder in current dir
sc.datasets.pbmc3k()

# Create a data description file to illustrate the scRNAseq data details 
data_des = "The data used in this basic preprocessing and clustering tutorial was collected from bone marrow mononuclear cells of healthy human donors. The samples used in this tutorial were measured using the 10X Multiome Gene Expression and Chromatin Accessability kit."

with open('data/data_description.txt','w') as f:
    f.write(data_des)
    
# Set workdir as current dir
file_config.WORK_DIR = current_dir

### Set up LLMs

In [ ]:
# Use Openai API for example, you can use other LLM provider as well
from langchain_openai import ChatOpenAI

# I will recommend use Openai API, I haven't tested any APIs from other suppliers yet
def call_chatllm_openai(key, api_base, model_name):
    llm = ChatOpenAI(
        api_key = key,
        base_url= api_base,
        model = model_name,
        temperature= 0,
        max_retries = 3,
        )
    return llm

# # Setup up api keys 
# openai_key = ""
# openai_base = ""
# openai_chat_model = "" 
# openai_code_model = ""

# Here, I recommend using an LLM with stronger coding capabilities as the code model, which is set to perform code-related work in all Ghostcoder graphs. Meanwhile, it is better to use LLM with stronger reasoning ability as the chat model.
chat_model = call_chatllm_openai(openai_key, openai_base, openai_chat_model)
code_model = call_chatllm_openai(openai_key, openai_base, openai_code_model)

### Set up Tavily search

In [ ]:
from ghostcoder.config import tavily_config
# Initial Tavily
# Currently, all web search functional is based on Tavily search, it's provide 10000 query/month free credit, more see https://app.tavily.com/home
tavily_api = ""

# You can set up Tavily by os environ
os.environ["TAVILY_API_KEY"] = tavily_api
# or with config, which work inside Ghostcoder only
tavily_config.TAVILY_API_KEY = tavily_api

# Set up max results of each Tavily query
tavily_config.MAX_RESULTS = 7

### File management and data perception by ghostcoder.filemanager

In [ ]:
# The fill manager will automatically set up file system and percept the input data 
# A initial file system status for a task with task_id should as follow: 
#─┬─────work_dir 
# ├─┬───data
# │ └───Input_data.whatever
# └─┬───task_id_1 // Work dir for every new tasks
#   ├─┬─data  
#   │ └─Input_data.whatever // A copy from work_dir/data/
#   ├───figures // Where output figures will be saved
#   └───results // Where processed data will be saved
from ghostcoder.config import docker_config, file_config
from ghostcoder.graph import create_filemanager_agent

#  Set workdir as current dir
current_dir = Path.cwd()
file_config.WORK_DIR = current_dir


# Initial graph
manager = create_filemanager_agent(
        chat_model = chat_model, 
        code_model = code_model,
        max_retry = 3,
        )

fm_input = {
    "task_id" : "Test", # 
    "docker_profile_dir": docker_config.DOCKER_PROFILES_DIR, # use pre-set docker profiles, please read those docker images 
    "max_iter": 10,
}

fm_state = await manager.ainvoke(fm_input)

In [ ]:
# Visualize the graph [optional], if failed try to run the cell again 
Image(manager.get_graph().draw_mermaid_png())

### Native env and Docker executor, test along

In [ ]:
from ghostcoder.graph import create_executor_agent
from ghostcoder.docker import get_docker_status
from ghostcoder.utils import get_native_env_perception

#  Set workdir as current dir
current_dir = Path.cwd()
file_config.WORK_DIR = current_dir

# Set up environment profiles
env_profiles = {
    "task_dirs":{
        "task_dir": "Test",
        "data_dir": "data",
        "figure_dir": "figures",
        "output_dir": "results",
    },
    "docker status": get_docker_status(),
    "native env languages": get_native_env_perception(),
}

# Initial agent
agent = create_executor_agent(
        chat_model = chat_model, 
        code_model = code_model,
        max_retry = 3,
    )



# Test executor with bash python and R
test_bash_code ="""
ls -al
"""

test_python_code = """
print("Hello World")
"""

test_r_code ="""
my_str <- "Hello World"
print(my_str)
"""

for codeblock in [
    test_bash_code, 
    test_python_code, 
    test_r_code
    ]:
    print("Test code executor with\n",codeblock)

    exe_states = await agent.ainvoke(
        {
            "generated_codeblock":codeblock,
            "env_profiles": env_profiles,
        }
    )

    print(f"Agent detected coding language as: {exe_states['language']}\nUse docker: {exe_states['use_docker']}")
    if exe_states['use_docker']:
        print(f"With docker image{exe_states['docker_image']}")
    print(f"Code execute output:\n{exe_states['execution_results']}\n--------\n\n")

In [ ]:
# Visualize the graph [optional], if failed try to run the cell again 
Image(agent.get_graph().draw_mermaid_png())

### Web crawler, test along

In [48]:
from ghostcoder.graph import create_crawler_agent

# Initial agent
agent = create_crawler_agent(
        chat_model = chat_model, 
        code_model = code_model,
        max_retry = 3,
    )

# Input 
query_context = "I'm looking for R codes for analysis single cell trajectory using monocl3"

# Invoke
crawl_state = agent.invoke(
    {
        "query_context": query_context,
    }
    )

# Print results
print("--------\nGenerated query for given context:")
for q in crawl_state['query_list']:
    print(q)
print(f"Get total {len(crawl_state['useful_results'])} useful web search results")
print("Crawled and parsed web information as follow:")
print(crawl_state['summary'])


Attempt 1: No content loaded.
Attempt 2: No content loaded.
Attempt 3: No content loaded.
Attempt 1: No content loaded.
Attempt 2: No content loaded.
Attempt 3: No content loaded.


INFO: HTTP Request: POST https://api3.aitok.ai/openai/v1/chat/completions "HTTP/1.1 200 OK"


--------
Generated query for given context:
R code for single cell trajectory analysis using Monocle3
Monocle3 single cell trajectory analysis R script
How to perform single cell trajectory analysis with Monocle3 in R
Get total 5 useful web search results
Crawled and parsed web information as follow:
The tutorial "Inferring single cell trajectories with Monocle3 (R)" provides a comprehensive guide on performing trajectory analysis using Monocle3 in R. It covers the transformation of an AnnData object into Monocle-compatible files, installation of necessary packages, and the Monocle3 workflow, including pre-processing, batch correction, dimensionality reduction, clustering, and trajectory inference. The tutorial also explains how to assign cell types to clusters using both supervised and unsupervised approaches and perform differential expression analysis. Key R code snippets from the tutorial include:

```r
# Install Bioconductor and some of its dependencies
if (!requireNamespace("Bioc

In [ ]:
# Visualize the graph [optional], if failed try to run the cell again 
Image(agent.get_graph().draw_mermaid_png())